#Fine-Tunning du modèle Resnet101


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Mettre les images en RGB

In [ ]:
import os
from PIL import Image

# Spécifiez le chemin de votre dossier d'images
dossier_images = "/content/drive/MyDrive/Projet_Vision/image_vision/random/val/0"
dossier_images2 = "/content/drive/MyDrive/Projet_Vision/image/val/0"

# Obtenez la liste de tous les fichiers dans le dossier
liste_fichiers = os.listdir(dossier_images)

# Boucle à travers tous les fichiers du dossier
for nom_fichier in liste_fichiers:
    chemin_image = os.path.join(dossier_images, nom_fichier)

    # Vérifiez si le fichier est une image (vous pouvez ajuster les extensions selon vos besoins)
    if nom_fichier.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
        # Chargez l'image en niveau de gris (L)
        image_en_rgb = Image.open(chemin_image).convert("RGB")

        # Enregistrez ou affichez l'image en RGB selon vos besoins
        # Par exemple, pour enregistrer l'image :
        chemin_image_en_rgb = os.path.join(dossier_images2, "RGB_" + nom_fichier)
        image_en_rgb.save(chemin_image_en_rgb)


#Traitement des images

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ])
}

#Affectation des différents dataset à leur dataloader respectives

In [ ]:
image_datasets = {
    'train':
    datasets.ImageFolder("/content/drive/MyDrive/Projet_Vision/image/train", data_transforms['train']),
    'validation':
    datasets.ImageFolder('/content/drive/MyDrive/Projet_Vision/image/val', data_transforms['validation']),
    'test':
    datasets.ImageFolder('/content/drive/MyDrive/Projet_Vision/image/test', data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=0),
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=0),
    'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=0)
}

In [ ]:
dataloaders["train"].dataset


Dataset ImageFolder
    Number of datapoints: 1600
    Root location: /content/drive/MyDrive/Projet_Vision/image/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
print(torch.cuda.is_available())


True


#Fine-Tunning du modèle et initialisation du modèle

In [ ]:
model = models.resnet101(weights = models.ResNet101_Weights.DEFAULT).to(device)

# Freeze all parameters by default
for param in model.parameters():
  param.requires_grad = True

# Specify layers to be trained
#layer1 = "layer4"
layer2 = "layer3"
layer3 = "layer2"
layer4 = "layer1"
for name, param in model.named_parameters():
  #if layer1 in name:
    #param.requires_grad = True
  if layer2 in name:
    param.requires_grad = False
  if layer3 in name:
    param.requires_grad = False
  if layer4 in name:
    param.requires_grad = False
  print(name, param.requires_grad)

model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)

Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to /root/.cache/torch/hub/checkpoints/resnet101-cd907fc2.pth
100%|██████████| 171M/171M [00:01<00:00, 154MB/s]


conv1.weight True
bn1.weight True
bn1.bias True
layer1.0.conv1.weight False
layer1.0.bn1.weight False
layer1.0.bn1.bias False
layer1.0.conv2.weight False
layer1.0.bn2.weight False
layer1.0.bn2.bias False
layer1.0.conv3.weight False
layer1.0.bn3.weight False
layer1.0.bn3.bias False
layer1.0.downsample.0.weight False
layer1.0.downsample.1.weight False
layer1.0.downsample.1.bias False
layer1.1.conv1.weight False
layer1.1.bn1.weight False
layer1.1.bn1.bias False
layer1.1.conv2.weight False
layer1.1.bn2.weight False
layer1.1.bn2.bias False
layer1.1.conv3.weight False
layer1.1.bn3.weight False
layer1.1.bn3.bias False
layer1.2.conv1.weight False
layer1.2.bn1.weight False
layer1.2.bn1.bias False
layer1.2.conv2.weight False
layer1.2.bn2.weight False
layer1.2.bn2.bias False
layer1.2.conv3.weight False
layer1.2.bn3.weight False
layer1.2.bn3.bias False
layer2.0.conv1.weight False
layer2.0.bn1.weight False
layer2.0.bn1.bias False
layer2.0.conv2.weight False
layer2.0.bn2.weight False
layer2.0.bn2.bi

#Paramètre utilisé

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.0001)

#Fonction pour l'entrainement du modèle

In [ ]:
import copy

def train_model(model, criterion, optimizer, num_epochs):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_loss = []
    val_loss = []
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            if phase == 'validation':
              val_loss.append(epoch_loss)
            if phase == 'train':
              train_loss.append(epoch_loss)
            # Deep copy the model if it has the best validation accuracy
            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    # Load best model weights
    model.load_state_dict(best_model_wts)

    return model, val_loss, train_loss

In [ ]:
model_trained, val_loss, train_loss = train_model(model, criterion, optimizer, num_epochs=10)

#Affichage de la loss

In [ ]:
epochs = range(1, len(val_loss) + 1)

# Plotting the loss values
plt.figure(figsize=(8, 6))
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

#Calcul de l'accuracy ainsi que de la matrice de convolution pour l'ensemble de test

In [ ]:
model = model_trained


In [ ]:
torch.save(model_trained.state_dict(), '/content/drive/MyDrive/Projet_Vision/weights10.h5')

In [ ]:
model = models.resnet101(weights = models.ResNet101_Weights.DEFAULT).to(device)

# Freeze all parameters by default
for param in model.parameters():
  param.requires_grad = True

# Specify layers to be trained
#layer1 = "layer4"
layer2 = "layer3"
layer3 = "layer2"
layer4 = "layer1"
for name, param in model.named_parameters():
  #if layer1 in name:
    #param.requires_grad = True
  if layer2 in name:
    param.requires_grad = False
  if layer3 in name:
    param.requires_grad = False
  if layer4 in name:
    param.requires_grad = False
  print(name, param.requires_grad)

model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/Projet_Vision/weights10.h5'))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

model.eval()

# Define variables for keeping track of accuracy and total tested samples
correct = 0
total = 0
true_labels = []
predicted_labels = []

class_1_counter = 0

# Iterate through the test dataset
for inputs, labels in dataloaders['test']:
    inputs = inputs.to(device)  # Send inputs to device (GPU or CPU)
    labels = labels.to(device)  # Send labels to device

    # Forward pass
    with torch.no_grad():  # No need to calculate gradients during inference
        outputs = model(inputs)

    _, predicted = torch.max(outputs.data, 1)
    print(predicted)

    true_labels.extend(labels.cpu().numpy())
    predicted_labels.extend(predicted.cpu().numpy())

    # Calculate accuracy
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

# Calculate overall accuracy
accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

# Generate the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Plot the confusion matrix
class_names = ['Class 0', 'Class 1']  # Replace with your actual class names
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()